<a href="https://colab.research.google.com/github/MadmanMarble/GLAP/blob/main/MidTermProject_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mid Term Project
##Description of the mid term project: this project implements linear regression from scratch with stochastic gradient descent, without using a library function except of course the necessary functions for array processing and autograd. The given code implements linear regression with the MSE loss function and the assignment is to experiment with other loss functions.
***Notes***: The linear regression code for this notebook is based on ***3.2. Linear Regression Implementation from Scratch*** of the [Dive into Deep Learning Book](https://d2l.ai/index.html)

In [ ]:
!pip install -U d2l

In [ ]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l


In [ ]:
def synthetic_data(w, b, num_examples):
    """Generate y = Xw + b + noise."""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [ ]:
print('features:', features[0], '\nlabel:', labels[0])

In [ ]:
d2l.set_figsize()
# The semicolon is for displaying the plot only
d2l.plt.scatter(features[:, 1].detach().numpy(),
                labels.detach().numpy(), 1);

In [ ]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i +
                                                   batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]


In [ ]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break


In [ ]:
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [ ]:
def linreg(X, w, b):
    """The linear regression model."""
    return torch.matmul(X, w) + b

In [ ]:
def squared_loss(y_hat, y):
    """Squared loss."""
    return (y_hat - y.reshape(y_hat.shape))**2 / 2

In [ ]:
def sgd(params, lr, batch_size):
    """Minibatch stochastic gradient descent."""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()


In [ ]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # Minibatch loss in `X` and `y`
        # Compute gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # Update parameters using their gradient
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

In [ ]:
print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
print(f'error in estimating b: {true_b - b}')

#Assigned tasks:
##The MSE loss function is not the only loss function for linear regression. To be robust to outliers in the training data, the Huber loss and the Log-Cosh are used. Your task is to implement both loss functions in addition to the given MSE loss function (named squared_loss) of this notebook, and to compare their performance on data with outliers. The specific tasks are as follows -
1.   Add two more code cells - one that defines the Huber loss function, and another with defines the Log-Cosh. This [blog post](https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0) describes these loss functions and provides code which is written in numpy so it will need to changed to compatible with PyTorch.
2.   Inject outliers in the data generated by the synthetic data generator of this notebook. You could add random noise to the existing data objects, i.e. changing a given percentage of the data entries to random values within the data range. Inclusion of these outliers will let you compare the benefits of these two loss functions against the MSE.
3.   Create a text cell to summarize your experimental observations as follows: (1) note how many outliers you added (2) compare the error in estimating w and b between MSE, Huber loss, and Log-Cosh loss.





